<a href="https://colab.research.google.com/github/michaelkigen/chocky/blob/main/emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 13.8 MB/s eta 0:00:00


In [39]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential #Fixed the typo here
from keras.layers import Dense, SimpleRNN, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [45]:
data = pd.read_csv('emoji_data.txt', header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [46]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])


In [47]:
X = data[0].values
Y = data[1].values
Y


array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 2, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2])

##Embedings

In [23]:
file = open('glove.6B.100d.txt', encoding='utf-8')
content = file.readlines()
file.close()


In [53]:
embedings = {}

for line in content:
  line = line.split()
  embedings[line[0]] = np.array(line[1:], dtype=float)



In [51]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index =  tokenizer.word_index


{'i': 1,
 'you': 2,
 'is': 3,
 'the': 4,
 'a': 5,
 'so': 6,
 'am': 7,
 'my': 8,
 'to': 9,
 'this': 10,
 'are': 11,
 'ha': 12,
 'for': 13,
 'she': 14,
 'he': 15,
 'me': 16,
 'not': 17,
 'love': 18,
 'your': 19,
 'want': 20,
 'have': 21,
 'it': 22,
 'got': 23,
 'like': 24,
 'did': 25,
 'baseball': 26,
 'food': 27,
 'was': 28,
 'do': 29,
 'joke': 30,
 'stop': 31,
 'will': 32,
 'miss': 33,
 'life': 34,
 'ball': 35,
 'good': 36,
 'what': 37,
 'go': 38,
 'job': 39,
 'funny': 40,
 'bad': 41,
 'day': 42,
 'great': 43,
 'dinner': 44,
 'that': 45,
 'with': 46,
 'at': 47,
 'of': 48,
 'game': 49,
 'we': 50,
 'again': 51,
 'said': 52,
 'yes': 53,
 'lol': 54,
 'and': 55,
 'down': 56,
 'had': 57,
 'her': 58,
 'fun': 59,
 'smile': 60,
 'lot': 61,
 'working': 62,
 'him': 63,
 'cute': 64,
 'on': 65,
 'lets': 66,
 'messing': 67,
 'us': 68,
 'play': 69,
 'exercise': 70,
 'lost': 71,
 'never': 72,
 'where': 73,
 'can': 74,
 'well': 75,
 'much': 76,
 'valentine': 77,
 'restaurant': 78,
 'awesome': 79,
 'lik

In [33]:
xtokens = tokenizer.texts_to_sequences(X)

In [36]:
def get_max_len(data):
  max_len = 0
  for i in data:
    max_len = max(max_len , len(i))
  return max_len

maxlen = get_max_len(xtokens)
maxlen

10

In [38]:
xtrain = pad_sequences(xtokens, maxlen=maxlen, padding='post', truncating='post')
xtrain

array([[103, 104,   3, ...,   0,   0,   0],
       [106,   3, 107, ...,   0,   0,   0],
       [  1,   7, 108, ...,   0,   0,   0],
       ...,
       [ 14,   3,   5, ...,   0,   0,   0],
       [ 14, 310,  26, ...,   0,   0,   0],
       [  1,  24,  22, ...,   0,   0,   0]], dtype=int32)

In [50]:
ytrain = to_categorical(Y)


##MOdel

In [56]:
embed_size = 100
embeding_matrix = np.zeros((len(word2index)+1, embed_size))

for word ,i in word2index.items():
  embed_vector = embedings[word]
  embeding_matrix[i] = embed_vector


In [58]:
embeding_matrix


array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [59]:
model = Sequential(
    [
        Embedding(
            input_dim=len(word2index)+1,
            output_dim=embed_size,
            input_length=maxlen,
            weights=[embeding_matrix],
            trainable=False
        ),
        LSTM(24, return_sequences=True),
        LSTM(8),
        Dense(5, activation='softmax')
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [60]:
model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.2747 - loss: 1.5905
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3628 - loss: 1.5496 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4219 - loss: 1.5084 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4546 - loss: 1.4809 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4224 - loss: 1.4441 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5349 - loss: 1.4011
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5651 - loss: 1.3606 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5741 - loss: 1.3225
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5954 - loss: 1.2864
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5586 - loss: 1.2322
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6291 - loss: 1.1612
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6094 - lo

In [61]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
I feel good ❤️
I feel very bad 😞
lets eat dinner 🍽️
